In [1]:
'''
https://scikit-learn.org/stable/modules/neural_networks_supervised.html#classification
https://scikit-learn.org/stable/supervised_learning.html


'''

'\nhttps://scikit-learn.org/stable/modules/neural_networks_supervised.html#classification\nhttps://scikit-learn.org/stable/supervised_learning.html\n\n\n'

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

#class
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from dmba import plotDecisionTree

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import PrecisionRecallDisplay

#score
from sklearn.metrics import precision_score, accuracy_score, roc_auc_score, confusion_matrix,f1_score, roc_curve, auc, recall_score

from sklearn.datasets import make_blobs
from sklearn.svm import LinearSVC



In [4]:
def evaluateBinaryClassification(predictions, actuals):
    contigency = pd.crosstab(actuals,predictions)
    TP = contigency[1][1]
    TN = contigency[0][0]
    FP = contigency[1][0]
    FN = contigency[0][1]
    n = contigency.sum().sum()

    Acuracy = (TP + TN)/n
    Recall = TP/(TP+FN)
    Precision = TP/(TP+FP)
    FScore = 2*Recall*Precision/(Recall+Precision)
    
    return Acuracy, Recall, Precision, FScore

#Read data

In [13]:
#Read data
df_mx = pd.read_csv("Sample Data/MX.csv", index_col=0)
df_thai = pd.read_csv("Sample Data/THAI.csv", index_col=0)
df_sgp = pd.read_csv("Sample Data/SGP.csv", index_col=0)

df_agg = pd.concat([df_thai, df_mx, df_sgp])
df_test = pd.concat([df_sgp])
df_agg.head()

,Short-term debt (% of total external debt),Real interest rate (%),Total reserves minus gold (current US$),Trade (% of GDP),Imports of goods and services (current US$),Exports of goods and services (current US$),Current account balance (% of GDP),"Portfolio investment, net (BoP, current US$)","Foreign direct investment, net inflows (% of GDP)",Real GDP (current US$),...,Left Government (democ),Polity (polity),Durable (durable),Turnover (xconst),Legislative/Executive election (exrec),State fragility index (sfi),Years (poleff),Economic Effectiveness (ecoeff),Presist (presistance),Target
"Total debt service (% of exports of goods, services and primary income)",,,,,,,,,,,,,,,,,,,,,
0.478220,0.417122,0.395807,0.000143,0.000000,0.000000,0.000000,0.156366,0.279665,0.212532,0.000014,...,0.4,0.60,1,3,8,0.319919,3.0,0.5,3,0
0.488071,0.418222,0.425714,0.000000,0.003768,0.000047,0.000350,0.147127,0.279731,0.202131,0.000000,...,0.0,0.15,0,1,4,0.319711,3.0,0.5,3,0
0.507164,0.417371,0.452184,0.000707,0.028019,0.000663,0.001292,0.158745,0.279695,0.244266,0.000427,...,0.0,0.15,1,1,4,0.319463,3.0,0.5,2,0
0.529773,0.418589,0.497835,0.001689,0.040052,0.002715,0.002906,0.150165,0.279506,0.227943,0.002647,...,0.0,0.00,0,0,0,0.319813,3.0,0.5,2,0
0.568958,0.417134,0.531526,0.003913,0.105083,0.006484,0.005783,0.151565,0.279147,0.316532,0.006558,...,0.4,0.65,0,3,8,0.320109,3.0,0.5,1,0


#Explore the Data

#Classification with Scikit-Learn

In [14]:
from sklearn import svm

# 讀入資料
X = df_agg.drop(columns=['Target'])
X = X.values
y = df_agg['Target'].values

# 切分訓練與測試資料
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [15]:
print(X,y)

[[4.17122388e-01 3.95807481e-01 1.43403000e-04 ... 3.00000000e+00
  5.00000000e-01 3.00000000e+00]
 [4.18221906e-01 4.25713738e-01 0.00000000e+00 ... 3.00000000e+00
  5.00000000e-01 3.00000000e+00]
 [4.17371226e-01 4.52183883e-01 7.07367000e-04 ... 3.00000000e+00
  5.00000000e-01 2.00000000e+00]
 ...
 [0.00000000e+00 6.01515596e-01 6.70284096e-01 ... 1.13333333e+00
  0.00000000e+00 5.30000000e+01]
 [0.00000000e+00 8.78230471e-01 8.69876486e-01 ... 1.36190476e+00
  0.00000000e+00 5.30000000e+01]
 [0.00000000e+00 1.67739517e-01 1.00000000e+00 ... 1.59047619e+00
  0.00000000e+00 5.30000000e+01]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0
 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]


In [16]:
# 建立分類器
clf = svm.SVC(kernel='rbf',C=30,gamma='auto')
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.9148936170212766

In [17]:
# 預測
y_test_predicted = clf.predict(X_test)
print(y_test_predicted)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 0 0 0 0]


In [18]:
# 標準答案
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0], dtype=int64)

In [27]:
# 績效


print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_predicted))
print("Accuracy:", accuracy_score(y_test, y_test_predicted))

print("Precision Score",precision_score(y_test, y_test_predicted, average='macro'))
print("Precision Score (Micro", precision_score(y_test, y_test_predicted, average='micro'))
print("Precision Score (Weighted)",precision_score(y_test, y_test_predicted, average='weighted'))
print("Precision Score", precision_score(y_test, y_test_predicted, average=None))

print("Recall Score (Macro)", recall_score(y_test, y_test_predicted, average='macro',zero_division=1))
print("Recall Score (Micro)", recall_score(y_test, y_test_predicted, average='micro',zero_division=1))
print("Recall Score (Weight)",recall_score(y_test, y_test_predicted, average='weighted',zero_division=1))
print("Recall Score (Avg None)",recall_score(y_test, y_test_predicted, average=None,zero_division=1))


print("Roc_Score:", roc_auc_score(y, clf.decision_function(X)))      
print("F1 Score (Macro):", f1_score(y_test, y_test_predicted, average='macro'))
print("F1 Score (Micro):", f1_score(y_test, y_test_predicted, average='micro'))
print("F1 Score (Weighted):", f1_score(y_test, y_test_predicted, average='weighted'))
print("F1 Score (Avg None):", f1_score(y_test, y_test_predicted, average=None))
print("F1 Score:", f1_score(y_test, y_test_predicted, zero_division=1))

Confusion Matrix:
 [[40  1]
 [ 3  3]]
Accuracy: 0.9148936170212766
Precision Score 0.8401162790697674
Precision Score (Micro 0.9148936170212766
Precision Score (Weighted) 0.9072241464621473
Precision Score [0.93023256 0.75      ]
Recall Score (Macro) 0.7378048780487805
Recall Score (Micro) 0.9148936170212766
Recall Score (Weight) 0.9148936170212766
Recall Score (Avg None) [0.97560976 0.5       ]
Roc_Score: 0.938340376488667
F1 Score (Macro): 0.7761904761904762
F1 Score (Micro): 0.9148936170212766
F1 Score (Weighted): 0.9073961499493416
F1 Score (Avg None): [0.95238095 0.6       ]
F1 Score: 0.6


Kfold Corss Validation

In [21]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='linear' ,C=10,gamma='auto'), X, y, cv=5)

array([0.40625   , 0.74193548, 0.48387097, 0.61290323, 0.83870968])

In [22]:
cross_val_score(svm.SVC(kernel='rbf' ,C=10,gamma='auto'), X, y, cv=5)

array([0.40625   , 0.74193548, 0.67741935, 0.90322581, 0.87096774])

In [23]:
cross_val_score(svm.SVC(kernel='linear' ,C=20,gamma='auto'), X, y, cv=5)

array([0.34375   , 0.74193548, 0.48387097, 0.61290323, 0.83870968])

In [26]:
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval ,C=cval,gamma='auto'), X, y, cv=5)
        avg_scores[kval+'_'+str(cval)] = np.average(cv_scores)
avg_scores

{'rbf_1': 0.8588709677419356,
 'rbf_10': 0.7199596774193548,
 'rbf_20': 0.7332661290322581,
 'linear_1': 0.6241935483870968,
 'linear_10': 0.616733870967742,
 'linear_20': 0.604233870967742}

GridSearch CV

In [28]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(X, y)
clf.cv_results_

{'mean_fit_time': array([0.00109663, 0.00254025, 0.00078173, 0.0099668 , 0.00058622,
        0.00840344]),
 'std_fit_time': array([0.00049987, 0.00132548, 0.00039086, 0.00434409, 0.00047865,
        0.00460775]),
 'mean_score_time': array([0.00096946, 0.00039077, 0.00058627, 0.00019536, 0.00019574,
        0.00039082]),
 'std_score_time': array([4.13699588e-05, 4.78591220e-04, 4.78688912e-04, 3.90720367e-04,
        3.91483307e-04, 4.78649591e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [29]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001097,0.000500,0.000969,0.000041,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.87500,0.903226,0.741935,0.903226,0.870968,0.858871,0.060025,1
1,0.002540,0.001325,0.000391,0.000479,1,linear,"{'C': 1, 'kernel': 'linear'}",0.25000,0.741935,0.451613,0.806452,0.870968,0.624194,0.235705,4
2,0.000782,0.000391,0.000586,0.000479,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.40625,0.741935,0.677419,0.903226,0.870968,0.719960,0.177255,3
3,0.009967,0.004344,0.000195,0.000391,10,linear,"{'C': 10, 'kernel': 'linear'}",0.40625,0.741935,0.483871,0.612903,0.838710,0.616734,0.159323,5
4,0.000586,0.000479,0.000196,0.000391,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.34375,0.806452,0.709677,0.935484,0.870968,0.733266,0.208561,2
5,0.008403,0.004608,0.000391,0.000479,20,linear,"{'C': 20, 'kernel': 'linear'}",0.34375,0.741935,0.483871,0.612903,0.838710,0.604234,0.176836,6


In [30]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.858871
1,1,linear,0.624194
2,10,rbf,0.719960
3,10,linear,0.616734
4,20,rbf,0.733266
5,20,linear,0.604234


In [ ]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_in_',
 'n_jobs',
 'n_splits

In [32]:
clf.best_score_

0.8588709677419356

In [35]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

randomizedSearchCV

In [36]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(X, y)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.624194
1,10,linear,0.616734


Differenct Hyperparameters

In [37]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [38]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.858871,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.858871,{'n_estimators': 10}
2,logistic_regression,0.745161,{'C': 1}


In [ ]:
#MLP